In [10]:
# import requests
# import pandas as pd

# API_KEY = "57307db9-c5ea-4863-99b5-17823e77489e"
# API_KEY = ["4fc06acb-3fe5-40b6-a8e7-9a4e8d845614","0ff9b2cf-abfb-4efe-a148-e30485cfe3cb"]

# # # def check_number_status(phone_number, api_keys=API_KEY):
# # #     url = "https://api.starsender.online/api/check-number"
# # #     payload = {"number": str(phone_number)}

# # #     for api_key in api_keys:
# # #         headers = {
# # #             "Content-Type": "application/json",
# # #             "Authorization": api_key
# # #         }
# # #         try:
# # #             response = requests.post(url, headers=headers, json=payload)
# # #             response.raise_for_status()
# # #             r = response.json()
# # #             return r.get("data", {}).get("message", "No message")
# # #         except requests.exceptions.HTTPError as e:
# # #             # If it's specifically a 400 error, try the next API key
# # #             if response.status_code == 400:
# # #                 continue
# # #             else:
# # #                 return f"HTTP Error: {str(e)}"
# # #         except requests.exceptions.RequestException as e:
# # #             return f"Error: {str(e)}"a api_key=API_KEY, timeout=60):
    # url = "https://api.starsender.online/api/check-number"
    # headers = {
    #     "Content-Type": "application/json",
    #     "Authorization": api_key
    # }
    # payload = {"number": str(phone_number)}

    # try:
    #     response = requests.post(url, headers=headers, json=payload, timeout=timeout)
    #     response.raise_for_status()
    #     r = response.json()
    #     return r.get("data", {}).get("message", "No message")
    # except requests.exceptions.Timeout:
    #     return "Timeout"
    # except requests.exceptions.RequestException as e:
    #     return f"Error: {str(e)}"
    

    # test
    # url = "https://api.starsender.online/api/check-number"
    # headers = {
    #     "Content-Type": "application/json",
    #     "Authorization": api_key
    # }
    # payload = {"number": str(phone_number)}

    # try:
    #     response = requests.post(url, headers=headers, json=payload, timeout=timeout)
    #     response.raise_for_status()
    #     r = response.json()
    #     return r.get("data", {}).get("message", "No message")
    # except requests.exceptions.Timeout:
    #     return "Timeout"
    # except requests.exceptions.RequestException as e:
    #     return f"Error: {str(e)}"


import requests
import pandas as pd
import time
import math

API_KEYS = [
    "468fd187-9460-42b3-adcb-934315fb3c60", 
    "f68ae405-a48a-432d-a4f8-1847ee3afd3c",
    "69969001-6dea-4fa0-8490-533cff7aabcf",
    "66715fc5-9def-486c-8cb7-c6c943833fd1",
    "4614076f-20c1-4632-941c-8a47dec5d386",
    "ba1e9466-0f1e-482a-a711-1c2ef8f455fd",
    "ba1e9466-0f1e-482a-a711-1c2ef8f455fd",
    "06b3535f-186f-4cb7-be6d-14ae08e4b48b"

]

def check_number_status(phone_number, api_key, timeout=60):
    url = "https://api.starsender.online/api/check-number"
    headers = {
        "Content-Type": "application/json",
        "Authorization": api_key
    }
    payload = {"number": str(phone_number)}

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()
        r = response.json()
        return r.get("data", {}).get("message", "No message")
    except requests.exceptions.Timeout:
        return "Timeout"
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"


def process_dataframe(df):
    total_rows = len(df)
    total_keys = len(API_KEYS)

    # Auto-calc batch size (ceil to avoid missing rows)
    batch_size = math.ceil(total_rows / total_keys)

    print(f"Total rows: {total_rows}")
    print(f"Total API keys: {total_keys}")
    print(f"Auto batch size = {batch_size}\n")

    batch_number = 1

    for i in range(total_rows):
        # Determine API key based on row index
        key_index = i // batch_size
        key_index = min(key_index, total_keys - 1)  # avoid overflow

        current_api_key = API_KEYS[key_index]
        phone_number = df.loc[i, "Whatsapp"]

        result = check_number_status(phone_number, current_api_key)
        df.loc[i, "validity"] = result

        print(f"Row {i+1}/{total_rows} | API {key_index+1}/{total_keys} | Num: {phone_number} → {result}")

        time.sleep(5)

        # Save per batch
        if (i + 1) % batch_size == 0:
            filename = f"batch_{batch_number}.xlsx"
            df.iloc[i + 1 - batch_size : i + 1].to_excel(filename, index=False)
            print(f"✅ Saved {filename}")

            batch_number += 1

            if batch_number <= total_keys:
                print("⏳ Waiting 10 minutes before switching to next API key...")
                time.sleep(10 * 60)

    # Save last remaining rows
    remaining = total_rows % batch_size
    if remaining > 0:
        filename = f"batch_{batch_number}.xlsx"
        df.iloc[-remaining:].to_excel(filename, index=False)
        print(f"✅ Saved {filename}")



# Example usage
# df = pd.read_excel("input.xlsx")
# process_dataframe(df)


In [2]:
# load data
database = pd.read_excel("Database.xlsx")

# load data validity (consist whatsapp number and the status registered or not)
data_with_validity = pd.read_excel("checkpoint_files/data_with_number_validity_updated.xlsx")



In [3]:
no_crm = database[database['CRM'].isna()]
uncheck = no_crm[(no_crm['validity'].isna()) & (no_crm['Source'] == 'Web Ads')].reset_index(drop=True)

In [5]:
process_dataframe(uncheck)

Processing 1/114 [Batch 1 | API 1]: 6281360022179 → Number registered
Processing 2/114 [Batch 1 | API 1]: 6285668420087 → Number registered
Processing 3/114 [Batch 1 | API 1]: 6281281738931 → Number registered
Processing 4/114 [Batch 1 | API 1]: 6285321311960 → Number registered
Processing 5/114 [Batch 1 | API 1]: 6281363520491 → Number registered
Processing 6/114 [Batch 1 | API 1]: 6281363405264 → Number registered
Processing 7/114 [Batch 1 | API 1]: 6282348425319 → Number registered
Processing 8/114 [Batch 1 | API 1]: 6282253337261 → Number registered
Processing 9/114 [Batch 1 | API 1]: 6281337765185 → Number registered
Processing 10/114 [Batch 1 | API 1]: 6285331638592 → Number registered
Processing 11/114 [Batch 1 | API 1]: 6282162771382 → Number registered
Processing 12/114 [Batch 1 | API 1]: 6285260515890 → Number registered
Processing 13/114 [Batch 1 | API 1]: 6285215490993 → Number registered
Processing 14/114 [Batch 1 | API 1]: 6282377531151 → Number registered
Processing 15/1

In [6]:
# process_dataframe(uncheck)
val_1 = pd.read_excel("batch_1.xlsx")
# val_2 = pd.read_excel("batch_2.xlsx")
# val_3 = pd.read_excel("batch_3.xlsx")

In [7]:
validity = pd.concat([val_1])
validity

,Unnamed: 0,Whatsapp,Donatur,Bulan,Tahun,CRM,Source,Total,Frekuensi,klasifikasi_program,Preferensi,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,validity
0,44778,6281360022179,Aisyah,November,2025,NaN,Web Ads,100960,1,NaN,1,05-06,100960,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
1,44779,6285668420087,JAMARI,November,2025,NaN,Web Ads,100393,1,NaN,1,08-09,100393,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
2,44780,6281281738931,Hamba Allah,November,2025,NaN,Web Ads,103641,1,NaN,1,15-16,103641,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
3,44781,6285321311960,Amithofo,November,2025,NaN,Web Ads,100947,1,NaN,1,06-07,100947,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
4,44782,6281363520491,Widya L,November,2025,NaN,Web Ads,53883,1,NaN,1,16-17,53883,Fri,Jumat,2025,Aktif,BRONZE,< 100.000,Number registered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,44887,6287719332613,hamba Allah,November,2025,NaN,Web Ads,100693,1,PALESTINA,1,18-19,100693,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
110,44888,628176724691,Hamroh,November,2025,NaN,Web Ads,100094,1,PALESTINA,1,20-21,100094,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
111,44889,6281286632200,Fadli Alhakim bin Catur Widya Andini,November,2025,NaN,Web Ads,50180,1,PALESTINA,1,14-15,50180,Sun,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Number not registered
112,44890,6285179808167,Muhammad Yusran,November,2025,NaN,Web Ads,500300,1,PALESTINA,1,13-14,500300,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered


In [8]:
validity_update = pd.concat([data_with_validity, validity])
validity_update = validity_update.drop_duplicates(subset=['Whatsapp'], keep='last').reset_index(drop=True)

In [9]:
validity_update.to_excel("checkpoint_files/data_with_number_validity_updated.xlsx")